# COIN Project

Authors:

    - Mona
    - Roman
    - Nick
    - Mateo

# Setup

## Packages

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import tweepy
import dotenv
import os
import yaml
import datetime
from pprint import pprint

## Environment variables

In [2]:
dotenv.load_dotenv()

True

In [3]:
with open("../twitter.yml", "r") as f:
    config = yaml.safe_load(f)

## Tweepy

In [ ]:
auth = tweepy.OAuth2BearerHandler(os.environ["BEARER_TOKEN"])
api = tweepy.API(auth)

In [4]:
auth = tweepy.OAuthHandler(os.environ["API_KEY"], os.environ["API_KEY_SECRET"])
auth.set_access_token(os.environ["ACCESS_TOKEN"], os.environ["ACCESS_TOKEN_SECRET"])
api = tweepy.API(auth)

## Database connection

In [ ]:
username = os.environ["DB_USERNAME"]
password = os.environ["DB_PASSWORD"]
host = os.environ["DB_HOST"]
port = os.environ["DB_PORT"]
name = os.environ["DB_NAME"]

In [ ]:
engine = sa.create_engine("mssql+pymssql://{}:{}@{}/{}".format(username, password, host, name))

# Twitter API

In [5]:
api.get_status(1520544762792607744, tweet_mode = "extended").full_text

"RT @Global_Witness: Hey @HSBC we're watching you! 👀\n\nYou're the UK’s biggest high street bank financing #deforestation. \n\nAs your sharehold…"

In [ ]:
tweet_list = api.user_timeline(screen_name="Fridays4future", tweet_mode="extended", count=10)

In [ ]:
tweet = tweet_list[0]

In [ ]:
tweet._json["id"]

In [ ]:
for tweet in tweepy.Cursor(api.user_timeline, id="Fridays4future", tweet_mode="extended").items(1):
    print(tweet.id)
    print(tweet.created_at)
    print(tweet.full_text)
    print(tweet)

In [ ]:
tweets = api.user_timeline(screen_name="Fridays4future", count=10)